# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [67]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

#Convert to Farenheit
city_data_df['Max Temp'] = city_data_df['Max Temp'].apply(lambda x: (x * 9/5) + 32)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,butaritari,3.0707,172.7902,81.626,81,8,7.68,KI,1675487657
1,1,bengkulu,-3.8004,102.2655,83.012,69,99,3.95,ID,1675487555
2,2,ust-kulom,61.6864,53.6902,11.354,99,99,1.61,RU,1675487658
3,3,rikitea,-23.1203,-134.9692,78.566,74,100,2.05,PF,1675487444
4,4,hobart,-42.8794,147.3294,60.260,92,75,8.23,AU,1675487532


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [43]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo = True,
    size="Humidity",
    hover_cols=["City"],
    color="City",
    tiles=True,
    alpha=0.3,
    width=900,
    height=500
)

# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [74]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df['Max Temp'] > 65) & (city_data_df['Max Temp'] < 80) & (city_data_df['Wind Speed'] < 10)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,rikitea,-23.1203,-134.9692,78.566,74,100,2.05,PF,1675487444
6,6,caravelas,-17.7125,-39.2481,75.884,88,5,3.20,BR,1675487659
7,7,bakel,14.9050,-12.4568,69.098,13,15,5.93,SN,1675487659
9,9,the valley,18.2170,-63.0578,78.098,75,8,7.90,AI,1675487660
13,13,puerto ayora,-0.7393,-90.3518,77.558,95,83,0.45,EC,1675487662


### Step 3: Create a new DataFrame called `hotel_df`.

In [77]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,rikitea,PF,-23.1203,-134.9692,74,
6,caravelas,BR,-17.7125,-39.2481,88,
7,bakel,SN,14.9050,-12.4568,13,
9,the valley,AI,18.2170,-63.0578,75,
13,puerto ayora,EC,-0.7393,-90.3518,95,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
# Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
sharanga - nearest hotel: Отель Юронга
tasiilaq - nearest hotel: Angmagssalik
aksu - nearest hotel: Akesu Tianyuan International Hotel
tomakomai - nearest hotel: グランドホテルニュー王子
erenhot - nearest hotel: Oulen hotel
nikolskoye - nearest hotel: No hotel found
pavlovsk - nearest hotel: Гостиница "Дон"
berlevag - nearest hotel: No hotel found
torbay - nearest hotel: Hampton
mayo - nearest hotel: The Westin
waynesville - nearest hotel: Waynesville Country Club Inn
hamilton - nearest hotel: North Vista Manor
la rioja - nearest hotel: No hotel found
sistranda - nearest hotel: Hotell Frøya
grants - nearest hotel: Southwest Motel
groa - nearest hotel: Sunndalsøra Hotell
korla - nearest hotel: Silver Star Hotel
wasilla - nearest hotel: No hotel found
havoysund - nearest hotel: Havøysund Hotell
mehamn - nearest hotel: Mehamn Arctic hotell
vardo - nearest hotel: Meieriet
ust-koksa - nearest hotel: Уймонская долина
petersburg - nearest hotel: Travel Inn - Petersburg
parfino - nea

,City,Country,Lat,Lng,Humidity,Hotel Name
11,sharanga,RU,57.1775,46.5396,97,Отель Юронга
19,tasiilaq,GL,65.6145,-37.6368,86,Angmagssalik
36,aksu,CN,41.1231,80.2644,73,Akesu Tianyuan International Hotel
74,tomakomai,JP,42.6369,141.6033,54,グランドホテルニュー王子
86,erenhot,CN,43.6475,111.9767,26,Oulen hotel
87,nikolskoye,RU,59.7035,30.7861,95,No hotel found
100,pavlovsk,RU,50.4578,40.1081,94,"Гостиница ""Дон"""
148,berlevag,NO,70.8578,29.0864,74,No hotel found
190,torbay,CA,47.6666,-52.7314,81,Hampton
201,mayo,US,38.8876,-76.5119,43,The Westin


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [82]:
%%capture --no-display

hotel_map = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    color="City",
    size="Humidity",
    hover_cols=["City", "Hotel Name", "Country"],
    tiles=True, 
    alpha=0.3,
)
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

In [101]:
## Here is another way to do this using plotly express

# Configure the map plot
hotel_map_alt = px.scatter_mapbox(hotel_df, 
                        lat="Lat", 
                        lon="Lng", 
                        #size="Humidity", 
                        text="Hotel Name", 
                        color="City", 
                        opacity=0.4, 
                        zoom=0)

hotel_map_alt.update_layout(mapbox_style="open-street-map")

# Display the map
hotel_map_alt.show()